In [1]:
import pandas as pd
import sqlite3 as sql


In [4]:
db_name = "stores.db"
conn = sql.connect(db_name)

In [5]:
query = """
    SELECT * FROM orderdetails
"""
pd.read_sql(query, conn)

,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4
...,...,...,...,...,...
2991,10425,S24_2300,49,127.79,9
2992,10425,S24_2840,31,31.82,5
2993,10425,S32_1268,41,83.79,11
2994,10425,S32_2509,11,50.32,6


In [8]:
query = """
    SELECT productcode, quantityOrdered
      FROM orderdetails
"""
pd.read_sql(query,conn).head(10)

,productCode,quantityOrdered
0,S18_1749,30
1,S18_2248,50
2,S18_4409,22
3,S24_3969,49
4,S18_2325,25
5,S18_2795,26
6,S24_1937,45
7,S24_2022,46
8,S18_1342,39
9,S18_1367,41


In [13]:
query = """
    SELECT productcode, sum(quantityOrdered) quantityOrdered
      FROM orderdetails
  GROUP BY productcode 
  ORDER BY 2 Desc
"""
pd.read_sql(query,conn)

,productCode,quantityOrdered
0,S18_3232,1808
1,S18_1342,1111
2,S700_4002,1085
3,S18_3856,1076
4,S50_1341,1074
...,...,...
104,S18_2870,855
105,S18_2248,832
106,S24_3969,824
107,S24_1046,803


In [15]:
query = """
    SELECT productCode , sum(quantityOrdered) quantityOrdered, (SELECT quantityInStock
                                                                  FROM products
                                                                 WHERE products.productCode = orderdetails.productCode) quantityInStock
      FROM orderdetails
   GROUP BY 1
   ORDER BY 2 Desc
"""
pd.read_sql(query,conn)

,productCode,quantityOrdered,quantityInStock
0,S18_3232,1808,8347
1,S18_1342,1111,8693
2,S700_4002,1085,8820
3,S18_3856,1076,2378
4,S50_1341,1074,7062
...,...,...,...
104,S18_2870,855,8164
105,S18_2248,832,540
106,S24_3969,824,2081
107,S24_1046,803,1005


# QUESTION : Which Products Should We Order More of or Less of?


## Low Stock(i.e. product in demand) = SUM (quantityordered)/quantityInStock


In [17]:
query = """
    SELECT productCode , sum(quantityOrdered)/(SELECT quantityInStock
                                                                  FROM products
                                                                 WHERE products.productCode = orderdetails.productCode)  low_stock
      FROM orderdetails
   GROUP BY 1
   ORDER BY 2 Desc
"""
pd.read_sql(query,conn)

,productCode,low_stock
0,S24_2000,67
1,S12_1099,13
2,S32_4289,7
3,S32_1374,5
4,S72_3212,2
...,...,...
104,S10_4757,0
105,S10_4698,0
106,S10_2016,0
107,S10_1949,0


In [18]:
query = """
    SELECT productCode , 
    Round(sum(quantityOrdered)*1.0/(SELECT quantityInStock
                                FROM products
                                WHERE products.productCode = orderdetails.productCode),2) low_stock                                          
      FROM orderdetails
   GROUP BY 1
   ORDER BY 2 Desc
      LIMIT 10
"""
pd.read_sql(query,conn)

,productCode,low_stock
0,S24_2000,67.67
1,S12_1099,13.72
2,S32_4289,7.15
3,S32_1374,5.70
4,S72_3212,2.31
5,S700_3167,1.90
6,S50_4713,1.65
7,S18_2795,1.61
8,S18_2248,1.54
9,S700_1938,1.22


# PRODUCT PERFORMANCE
###                product performance = SUM(quantityOrdered * priceEach).

In [19]:
query = """
    SELECT productCode, sum(quantityOrdered*priceEach) prod_perf
     From orderdetails
     GROUP BY 1
     ORDER BY 2 Desc
     LIMIT 10"""
pd.read_sql(query,conn)

,productCode,prod_perf
0,S18_3232,276839.98
1,S12_1108,190755.86
2,S10_1949,190017.96
3,S10_4698,170686.00
4,S12_1099,161531.48
5,S12_3891,152543.02
6,S18_1662,144959.91
7,S18_2238,142530.63
8,S18_1749,140535.60
9,S12_2823,135767.03


In [20]:
#Common Table Expression (CTE)
query = """With
low_stock_table AS (
SELECT productCode,
       ROUND(SUM(quantityOrdered)*1.0/(SELECT quantityInStock
                                FROM products
                                WHERE products.productCode = orderdetails.productCode),2) low_stock                                          
      FROM orderdetails
   GROUP BY 1
   ORDER BY 2 Desc
      LIMIT 10
)
SELECT productCode FROM low_stock_table;
"""
pd.read_sql(query,conn)       

,productCode
0,S24_2000
1,S12_1099
2,S32_4289
3,S32_1374
4,S72_3212
5,S700_3167
6,S50_4713
7,S18_2795
8,S18_2248
9,S700_1938


In [21]:
#priority products for restocking
query = """With
low_stock_table AS (
SELECT productCode,
       ROUND(SUM(quantityOrdered)*1.0/(SELECT quantityInStock
                                FROM products
                                WHERE products.productCode = orderdetails.productCode),2) low_stock                                          
      FROM orderdetails
   GROUP BY 1
   ORDER BY 2 Desc
      LIMIT 10
)
SELECT productCode, sum(quantityOrdered*priceEach) prod_perf
     From orderdetails
     Where productCode IN(SELECT productCode FROM low_stock_table) 
     GROUP BY 1
     ORDER BY 2 Desc
     ;
"""
pd.read_sql(query,conn)


,productCode,prod_perf
0,S12_1099,161531.48
1,S18_2795,132275.98
2,S32_1374,89364.89
3,S700_3167,76618.40
4,S50_4713,73670.64
5,S700_1938,69531.61
6,S24_2000,67193.49
7,S32_4289,60493.33
8,S72_3212,47550.40
9,S18_2248,45306.77
